In [6]:
import ray 
import os

def stop_clark_cluster():
    ray.shutdown()
    os.system('ssh ellen /home/john/venv/bin/ray stop')
    os.system('ssh audrey /home/john/venv/bin/ray stop')
    os.system('ssh edna /home/john/venv/bin/ray stop')
    
def start_clark_cluster(clark_cpus = 4, ellen_cpus = 32, audrey_cpus = 32, edna_cpus = 32, 
                        clark_gpus = 0, ellen_gpus = 2, audrey_gpus = 1, edna_gpus = 1):
    stop_clark_cluster()
    config = ray.init(redis_port=6379, redis_password="gn8GWVrMJ3cSX4", num_cpus = clark_cpus, num_gpus = clark_gpus)
    os.system("ssh ellen ray start --address={} --redis-password='gn8GWVrMJ3cSX4' --num-gpus={} --num-cpus={}".format(config['redis_address'], ellen_gpus, ellen_cpus))
    os.system("ssh edna ray start --address={} --redis-password='gn8GWVrMJ3cSX4' --num-gpus={} --num-cpus={}".format(config['redis_address'], edna_gpus, edna_cpus))
    os.system("ssh audrey ray start --address={} --redis-password='gn8GWVrMJ3cSX4' --num-gpus={} --num-cpus={}".format(config['redis_address'], audrey_gpus, audrey_cpus))
    return config

In [7]:
stop_clark_cluster()

In [ ]:
ray.init(address='auto')
ray.nodes()

In [34]:
import ray
ray.__version__
ray.shutdown()

In [4]:
password = 'C4$@#jNSqY9jJh'
port = 6379
n_cpus = None
n_gpus = None

ray.init(redis_port=port, redis_password=password, num_cpus=n_cpus, num_gpus=n_gpus)


2020-12-05 09:46:25,961	INFO resource_spec.py:212 -- Starting Ray with 4.35 GiB memory available for workers and up to 2.17 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-12-05 09:46:26,223	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.251',
 'redis_address': '192.168.1.251:6379',
 'object_store_address': '/tmp/ray/session_2020-12-05_09-46-25_949190_25050/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-05_09-46-25_949190_25050/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-12-05_09-46-25_949190_25050'}

# distributed_computing module example

In [1]:
import pipecaster as pc
cluster = pc.HeadNode(n_cpus=4, n_gpus=0, object_store_memory='auto')
cluster.connect_remote_node('ellen', ray_dir='/home/john/venv/bin/',
                       n_cpus = 32, n_gpus = 2, object_store_memory='auto')
cluster.connect_remote_node('audrey', ray_dir='/home/john/venv/bin/',
                       n_cpus = 32, n_gpus = 1, object_store_memory='auto')
cluster.connect_remote_node('edna', ray_dir='/home/john/venv/bin/',
                       n_cpus = 32, n_gpus = 1, object_store_memory='auto')

2020-12-07 08:51:57,517	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


In [2]:
# wait ~1 second for resources to be update, then you can call
print('resources: {} CPUs and {} GPUS'.format(cluster.count_cpus(), cluster.count_gpus()))

resources: 100.0 CPUs and 4.0 GPUS


In [3]:
cluster.restart()

2020-12-07 08:52:11,617	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


In [5]:
# wait ~1 second for resources to be update, then you can call
print('resources: {} CPUs and {} GPUS'.format(cluster.count_cpus(), cluster.count_gpus()))

resources: 100.0 CPUs and 4.0 GPUS


# from scratch

In [7]:
import ray
import os

config = ray.init()

2020-12-12 09:02:07,252	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


In [8]:
os.system("ssh ellen /home/john/venv/bin/ray start --address={} --num-gpus={} --num-cpus={}".format(config['redis_address'], 2, 32))

0

In [9]:
os.system("ssh audrey /home/john/venv/bin/ray start --address={} --num-gpus={} --num-cpus={}".format(config['redis_address'], 1, 32))

0

In [10]:
os.system("ssh edna /home/john/venv/bin/ray start --address={} --num-gpus={} --num-cpus={}".format(config['redis_address'], 1, 32))

0

In [16]:
import time
import numpy as np

def f(a, bigarg):
    x = 0
    for i in range(100000000):
        x += 1  
    return a

ray_f = ray.remote(f)

In [18]:
import time

t = time.time()

bigarg = np.random.rand(1, 1)
args_list = [(a, bigarg) for a in range(100)]
jobs = [ray_f.remote(*args) for args in args_list]
result = ray.get(jobs)

print('elapsed time = {:0.3} s'.format(time.time() - t))

(pid=gcs_server) E1212 09:09:14.853342000 123145336164352 tcp_server_posix.cc:213]      Failed accept4: Too many open files


elapsed time = 12.9 s


In [21]:
import time

n_jobs = 100
chunk_size = 25

t = time.time()

bigarg = np.random.rand(1, 1)
args_list = [(a, bigarg) for a in range(n_jobs)]
args_chunks = [args_list[i : i + chunk_size] for i in range(0, n_jobs, chunk_size)]

results = []

for args_chunk in args_chunks:
    job_chunk = [ray_f.remote(*args) for args in args_chunk]
    results.extend(ray.get(job_chunk))
    
print('number of results = {}'.format(len(results)))

print('elapsed time = {:0.3} s'.format(time.time() - t))

number of results = 100
elapsed time = 33.7 s


In [20]:
import time

n_jobs = 100
chunk_size = 25

t = time.time()

bigarg = np.random.rand(1, 1)
args_list = [(a, bigarg) for a in range(n_jobs)]
jobs = [ray_f.remote(*args) for args in args_list]
job_chunks = [jobs[i : i + chunk_size] for i in range(0, n_jobs, chunk_size)]

results = []
for job_chunk in job_chunks:
    results.extend(ray.get(job_chunk))
    
print('number of results = {}'.format(len(results)))

print('elapsed time = {:0.3} s'.format(time.time() - t))

number of results = 100
elapsed time = 11.1 s


In [10]:
import time

t = time.time()

a = bigarg

jobs = [ray_f.remote(a,b) for b in range(100)]
result = ray.get(jobs)

print('elapsed time = {:0.3} s'.format(time.time() - t))

elapsed time = 60.3 s


In [12]:
import time

t = time.time()

a = ray.put(bigarg)
args_list = [(a, b) for b in range(100)]

jobs = [ray_f.remote(*args) for args in args_list]
result = ray.get(jobs)

print('elapsed time = {:0.3} s'.format(time.time() - t))

elapsed time = 2.35 s


In [ ]:
from ray.util.multiprocessing import Pool
pool = Pool(processes=4, ray_address='auto')

In [9]:
del pool

In [15]:
del pool

In [ ]:
jobs = [ray_f.remote(*a) for a in args]
result = ray.get(jobs)
result

In [6]:
import time
import numpy as np

@ray.remote
def f(bigarg, index):
    x = 0
    for i in range(100000):
        x += 1  
    return index

In [112]:
bigarg = np.random.rand(300,100)
bigarg = ray.put(bigarg)
jobs = [f.remote(bigarg, i) for i in range(100)]
result = ray.get(jobs)
np.array(result)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [2]:
import numpy as np

def is_in(obj, iterable):
    for element in iterable:
        if element is obj:
            return True
    return False

def get_index(obj, iterable):
    index = 0
    for element in iterable:
        if element is obj:
            return index
        index += 1
    raise ValueError('object not found in the iterable')

def map_distributed(shared_mem_objects, f, *arg_lists):
    if shared_mem_objects is not None:
        plasma_objects = [ray.put(obj) for obj in shared_mem_objects]
        arg_lists = [[plasma_objects[get_index(obj, shared_mem_objects)] if is_in(obj, shared_mem_objects) 
                      else obj for obj in arg_list] for arg_list in arg_lists]
    
    args_list = list(zip(*arg_lists))
    jobs = [f.remote(*args) for args in args_list]
    return ray.get(jobs)

@ray.remote
def f(bigarg, index):
    x = 0
    for i in range(100000000):
        x += 1  
    return index

In [11]:
import importlib
importlib.reload(parallel)

<module 'pipecaster.parallel' from '/Users/john/trading/src/pipecaster/pipecaster/parallel.py'>

In [13]:
import numpy as np
import ray

def f(bigarg, index):
    x = 0
    for i in range(1000000000):
        x += 1  
    return index

In [15]:
import pipecaster.parallel as parallel
bigarg = np.random.rand(300, 20000)
n_jobs = 50
%time parallel.map_jobs(50, [bigarg], f, [bigarg for i in range(n_jobs)], range(n_jobs))

using ray.spool.startmap()
CPU times: user 1.93 s, sys: 740 ms, total: 2.67 s
Wall time: 1min 37s


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [2]:
import numpy as np

def is_in(obj, iterable):
    for element in iterable:
        if element is obj:
            return True
    return False

def get_index(obj, iterable):
    index = 0
    for element in iterable:
        if element is obj:
            return index
        index += 1
    raise ValueError('object not found in the iterable')

def map_distributed(shared_mem_objects, f, *arg_lists):
    if shared_mem_objects is not None:
        plasma_objects = [ray.put(obj) for obj in shared_mem_objects]
        arg_lists = [[plasma_objects[get_index(obj, shared_mem_objects)] if is_in(obj, shared_mem_objects) 
                      else obj for obj in arg_list] for arg_list in arg_lists]
    
    args_list = list(zip(*arg_lists))
    jobs = [f.remote(*args) for args in args_list]
    return ray.get(jobs)

@ray.remote
def f(bigarg, index):
    x = 0
    for i in range(100000000):
        x += 1  
    return index

In [ ]:
import pipecaster.parallel as parallel
bigarg = np.random.rand(300, 5000)
n_jobs = 99
%timeit parallel.map_jobs(n_jobs, [bigarg], f, [bigarg for i in range(n_jobs)], range(n_jobs))

In [22]:
bigarg = np.random.rand(1, 1)
n_jobs = 100
%timeit map_distributed([bigarg], f, [bigarg for i in range(n_jobs)], range(n_jobs))

11.1 s ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit map_distributed(None, f, [bigarg for i in range(n_jobs)], range(n_jobs))

11.2 s ± 43.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
bigarg = np.random.rand(100, 10000)
n_jobs = 100
%timeit map_distributed([bigarg], f, [bigarg for i in range(n_jobs)], range(n_jobs))

13.1 s ± 70.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit map_distributed(None, f, [bigarg for i in range(n_jobs)], range(n_jobs))

1min 10s ± 287 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
mylist = [1, bigarg]

0

In [11]:
import ray
import numpy as np

def f(bigarg, index):
    x = 0
    for i in range(1000000):
        x += 1  
    return index

ray_f = ray.remote(f)

In [13]:
bigarg = np.random.rand(100, 100)
plasma_bigarg = ray.put(bigarg)
jobs = [ray_f.remote(plasma_bigarg, i) for i in range(100)]
ray.get(jobs)

KeyboardInterrupt: 

In [1]:
from pipecaster.testing_utils import DummyClassifier
import numpy as np


clf = DummyClassifier()

In [2]:
clf.fit(np.random.rand(100,200), np.random.choice(['a','b'], 100))

In [5]:
%debug

> /Users/john/trading/src/pipecaster/mtrand.pyx(4065)numpy.random.mtrand.RandomState.shuffle()

ipdb> l

ipdb> l

ipdb> list

ipdb> y_enc
*** NameError: name 'y_enc' is not defined
ipdb> locals().keys()
dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__builtins__', 'RandomState', 'operator', 'warnings', 'np', '_integers_types', '_MT19937', '_rand', 'beta', 'binomial', 'bytes', 'chisquare', 'choice', 'dirichlet', 'exponential', 'f', 'gamma', 'get_state', 'geometric', 'gumbel', 'hypergeometric', 'laplace', 'logistic', 'lognormal', 'logseries', 'multinomial', 'multivariate_normal', 'negative_binomial', 'noncentral_chisquare', 'noncentral_f', 'normal', 'pareto', 'permutation', 'poisson', 'power', 'rand', 'randint', 'randn', 'random', 'random_integers', 'random_sample', 'rayleigh', 'seed', 'set_state', 'shuffle', 'standard_cauchy', 'standard_exponential', 'standard_gamma', 'standard_normal', 'standard_t', 'triangular', 'uniform', 'vonmises', 'wald', '

In [6]:
import pipecaster.parallel as parallel

def f(a, b):
    return a + b
    
As = [1, 2, 3]
Bs = [4, 5, 6]
    
results = parallel.map_jobs(f, As, Bs, n_jobs='max')

running max processes, 3 jobs using ray.get


In [7]:
results

[5, 7, 9]

In [8]:
args_list = zip(As, Bs)
results = parallel.starmap_jobs(f, args_list, n_jobs='max')

running max processes, 3 jobs using ray.get


In [10]:
print(results)

[5, 7, 9]


In [2]:
import os

defaul_redis_port = 6379

command= 'ray start --redis-password={} --redis-port={}'.format('gn8GWVrMJ5cSX4',6380)
os.system(command)

512

In [ ]:
import ray

ray.init(address='auto')

2020-12-12 06:26:21,881	INFO worker.py:651 -- Connecting to existing Ray cluster at address: 192.168.1.251:6379


In [1]:
import ray
ray.is_initialized()

False

In [2]:
import os
command= 'ray stop'
os.system(command)

0

# shutdown cluster

In [6]:
import ray
import os
ray.shutdown()
os.system("ssh edna /home/john/venv/bin/ray stop")
os.system("ssh ellen /home/john/venv/bin/ray stop")
os.system("ssh audrey /home/john/venv/bin/ray stop")

0

In [1]:
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_classification

import pipecaster as pc

X_rand = np.random.rand(500,30)
X_inf, y = make_classification(n_samples=500, n_features=30, 
                               n_informative=15, class_sep=1, random_state=None)

Xs = [X_rand, X_inf, X_inf, X_rand, X_rand, X_inf]

clf = pc.MultichannelPipeline(n_channels=6)

clf.add_layer(SimpleImputer())
clf.add_layer(StandardScaler())
clf.add_layer(SelectPercentile(percentile=25))
clf.add_layer(5, pc.SelectKBestScores(feature_scorer=f_classif, aggregator=np.mean, k=2)) 
predictors = [KNeighborsClassifier() for i in range(5)]
clf.add_layer(5, pc.SelectKBestModels(predictors=predictors, k=1), 1, LogisticRegression()) 
clf.add_layer(pc.MultichannelPredictor(SVC()))

clf.fit(Xs, y)
clf.predict(Xs)

> /Users/john/trading/src/pipecaster/pipecaster/channel_selection.py(129)__init__()
-> estimator_types = [p._estimator_type for p in predictors]
(Pdb) l
124  	        if predictors is None:
125  	            raise ValueError('No predictors found.')
126  	
127  	        if isinstance(predictors, (list, tuple, np.ndarray)):
128  	            import pdb; pdb.set_trace()
129  ->	            estimator_types = [p._estimator_type for p in predictors]
130  	        else:
131  	            estimator_types = [predictors._estimator_type]
132  	        if len(set(estimator_types)) != 1:
133  	            raise TypeError('Predictors must be of uniform type (e.g. all classifiers or all regressors).')
134  	        self._estimator_type = estimator_types[0]
(Pdb) predictors
[KNeighborsClassifier(), KNeighborsClassifier(), KNeighborsClassifier(), KNeighborsClassifier(), KNeighborsClassifier()]
(Pdb) c
> /Users/john/trading/src/pipecaster/pipecaster/channel_selection.py(129)__init__()
-> estimator_types

BdbQuit: 

In [5]:
scores = np.array([1,2,None])
scores[scores == None] = np.nan
k=1
np.argsort(scores)[-k:]

array([2])